In [1]:
import tensorflow as tf
import numpy as np

# 1、概念描述

### 1.1、Tensor

Tensor的意思是张量，不过按我的理解，其实就是指矩阵。也可以理解为tensorflow中矩阵的表示形式。Tensor的生成方式有很多种，最简单的就如

In [2]:
a = tf.zeros(shape=[1,2])

不过要注意，因为在训练开始前，所有的数据都是抽象的概念，也就是说，此时a只是表示这应该是个1*2的零矩阵，而没有实际赋值，也没有分配空间，所以如果此时print,就会出现如下情况:

In [3]:
print (a)

Tensor("zeros:0", shape=(1, 2), dtype=float32)


只有在训练过程开始后，才能获得a的实际值

In [4]:
sess = tf.InteractiveSession()
print(sess.run(a))

[[ 0.  0.]]


### 1.2、 Variable

故名思议，是变量的意思。一般用来表示图中的各计算参数，包括矩阵，向量等。例如，我要表示上图中的模型，那表达式就是 


                                y=Relu(Wx+b)

 （relu是一种激活函数，具体可见这里）这里W和b是我要用来训练的参数，那么此时这两个值就可以用Variable来表示。Variable的初始函数有很多其他选项，这里先不提，只输入一个Tensor也是可以的

In [5]:
W = tf.Variable(tf.zeros(shape=[1,2]))

In [6]:
print (W)

<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32_ref>


注意，此时W一样是一个抽象的概念，而且与Tensor不同，Variable必须初始化以后才有具体的值。

In [7]:
tensor = tf.zeros(shape=[1,2])
variable = tf.Variable(tensor)
sess = tf.InteractiveSession()
# print(sess.run(variable))  # 会报错
sess.run(tf.initialize_all_variables()) # 对variable进行初始化
print(sess.run(variable))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 0.  0.]]


### 1.3、placeholder

又叫占位符，同样是一个抽象的概念。用于表示输入输出数据的格式。告诉系统：这里有一个值/向量/矩阵，现在我没法给你具体数值，不过我正式运行的时候会补上的！例如上式中的x和y。因为没有具体数值，所以只要指定尺寸即可

In [8]:
x = tf.placeholder(tf.float32,[1, 5],name='input')
y = tf.placeholder(tf.float32,[None, 5],name='input')

上面有两种形式，第一种x，表示输入是一个[1,5]的横向量。 
 而第二种形式，表示输入是一个[?,5]的矩阵。那么什么情况下会这么用呢?就是需要输入一批[1,5]的数据的时候。比如我有一批共10个数据，那我可以表示成[10,5]的矩阵。如果是一批5个，那就是[5,5]的矩阵。tensorflow会自动进行批处理

### 1.4、Session

session，也就是会话。我的理解是，session是抽象模型的实现者。为什么之前的代码多处要用到session？因为模型是抽象的嘛，只有实现了模型以后，才能够得到具体的值。同样，具体的参数训练，预测，甚至变量的实际值查询，都要用到session,看后面就知道了

### 1.5、TFRecords

关于Tensorflow读取数据，官网给出了三种方法：
•供给数据(Feeding)： 在TensorFlow程序运行的每一步， 让Python代码来供给数据。
•从文件读取数据： 在TensorFlow图的起始， 让一个输入管线从文件中读取数据。
•预加载数据： 在TensorFlow图中定义常量或变量来保存所有数据(仅适用于数据量比较小的情况)。

对于数据量较小而言，可能一般选择直接将数据加载进内存，然后再分batch输入网络进行训练（tip:使用这种方法时，结合yield 使用更为简洁，大家自己尝试一下吧，我就不赘述了）。但是，如果数据量较大，这样的方法就不适用了，因为太耗内存，所以这时最好使用tensorflow提供的队列queue，也就是第二种方法 从文件读取数据。对于一些特定的读取，比如csv文件格式，官网有相关的描述，在这儿我介绍一种比较通用，高效的读取方法（官网介绍的少），即使用tensorflow内定标准格式——TFRecords


TFRecords其实是一种二进制文件，虽然它不如其他格式好理解，但是它能更好的利用内存，更方便复制和移动，并且不需要单独的标签文件（等会儿就知道为什么了）… …总而言之，这样的文件格式好处多多，所以让我们用起来吧。

TFRecords文件包含了tf.train.Example 协议内存块(protocol buffer)(协议内存块包含了字段 Features)。我们可以写一段代码获取你的数据， 将数据填入到Example协议内存块(protocol buffer)，将协议内存块序列化为一个字符串， 并且通过tf.python_io.TFRecordWriter 写入到TFRecords文件。

从TFRecords文件中读取数据， 可以使用tf.TFRecordReader的tf.parse_single_example解析器。这个操作可以将Example协议内存块(protocol buffer)解析为张量。


#### 生成TFRecords文件

我们使用tf.train.Example来定义我们要填入的数据格式，然后使用tf.python_io.TFRecordWriter来写入。


In [ ]:
import os
import tensorflow as tf 
from PIL import Image

cwd = os.getcwd()

'''
此处我加载的数据目录如下：
-- img1.jpg
     img2.jpg
     img3.jpg
     ...
-- img1.jpg
     img2.jpg
     ...
-- ...
...
'''
writer = tf.python_io.TFRecordWriter("train.tfrecords")
for index, name in enumerate(classes):
    class_path = cwd + name + "/"
    for img_name in os.listdir(class_path):
        img_path = class_path + img_name
            img = Image.open(img_path)
            img = img.resize((224, 224))
        img_raw = img.tobytes()              #将图片转化为原生bytes
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        }))
        writer.write(example.SerializeToString())  #序列化为字符串
writer.close()

关于Example Feature的相关定义和详细内容，我推荐去官网查看相关API。

基本的，一个Example中包含Features，Features里包含Feature（这里没s）的字典。最后，Feature里包含有一个 FloatList， 或者ByteList，或者Int64List

就这样，我们把相关的信息都存到了一个文件中，所以前面才说不用单独的label文件。而且读取也很方便。


In [ ]:
for serialized_example in tf.python_io.tf_record_iterator("train.tfrecords"):
    example = tf.train.Example()
    example.ParseFromString(serialized_example)

    image = example.features.feature['image'].bytes_list.value
    label = example.features.feature['label'].int64_list.value
    # 可以做一些预处理之类的
    print image, label

### 使用队列读取

一旦生成了TFRecords文件，接下来就可以使用队列（queue）读取数据了。

In [ ]:
def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [224, 224, 3])
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5
    label = tf.cast(features['label'], tf.int32)

    return img, label

In [ ]:
img, label = read_and_decode("train.tfrecords")

#使用shuffle_batch可以随机打乱输入
img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                batch_size=30, capacity=2000,
                                                min_after_dequeue=1000)
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    threads = tf.train.start_queue_runners(sess=sess)  #启动多线程处理数据
    for i in range(3):
        val, l= sess.run([img_batch, label_batch])
        #我们也可以根据需要对val， l进行处理
        #l = to_categorical(l, 12) 
        print(val.shape, l)

还有几个注意事项：

第一，tensorflow里的graph能够记住状态（state），这使得TFRecordReader能够记住tfrecord的位置，并且始终能返回下一个。而这就要求我们在使用之前，必须初始化整个graph，这里我们使用了函数tf.initialize_all_variables()来进行初始化。

第二，tensorflow中的队列和普通的队列差不多，不过它里面的operation和tensor都是符号型的（symbolic），在调用sess.run()时才执行。

第三， TFRecordReader会一直弹出队列中文件的名字，直到队列为空。


# 2 模型构建

我们使用官方tutorial中的mnist数据集的分类代码，公式可以写作 
                          
                          z=Wx+b
                          a=softmax(z)

In [9]:
# 建立抽象模型
x = tf.placeholder(tf.float32, [None, 784]) # 输入占位符
y = tf.placeholder(tf.float32, [None, 10])  # 输出占位符（预期输出）
W = tf.Variable(tf.zeros([784, 10]))        
b = tf.Variable(tf.zeros([10]))
a = tf.nn.softmax(tf.matmul(x, W) + b)      # a表示模型的实际输出

# 定义损失函数和训练方法
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y*tf.log(a), reduction_indices=[1])) # 损失函数为交叉熵
NEAR_0=1e-10
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y*tf.log(a+NEAR_0)+(1-y)*tf.log(1-a+NEAR_0), reduction_indices=[1])) # 损失函数为另一交叉熵
optimizer = tf.train.GradientDescentOptimizer(0.5) # 梯度下降法，学习速率为0.5
train = optimizer.minimize(cross_entropy)  # 训练目标：最小化损失函数

可以看到这样以来，模型中的所有元素(图结构，损失函数，下降方法和训练目标)都已经包括在train里面。我们可以把train叫做训练模型。那么我们还需要测试模型

In [10]:
correct_prediction = tf.equal(tf.argmax(a, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [71]:
en1=np.array([[1,2,3],[2,3,4]])
en1=en1.astype(np.float32)
print (en1)
en2=tf.log(en1)

[[ 1.  2.  3.]
 [ 2.  3.  4.]]


In [72]:
sess1 = tf.InteractiveSession()
print(sess1.run(en2))

[[ 0.          0.69314718  1.09861231]
 [ 0.69314718  1.09861231  1.38629436]]


In [74]:
1-en1

array([[ 0., -1., -2.],
       [-1., -2., -3.]], dtype=float32)

上述两行代码，tf.argmax表示找到最大值的位置(也就是预测的分类和实际的分类)，然后看看他们是否一致，是就返回true,不是就返回false,这样得到一个boolean数组。tf.cast将boolean数组转成int数组，最后求平均值，得到分类的准确率(怎么样，是不是很巧妙)

# 3 实际训练

有了训练模型和测试模型以后，我们就可以开始进行实际的训练了

In [12]:
from tensorflow.examples.tutorials.mnist import input_data

In [13]:
mnist=input_data.read_data_sets("E:/machine_data/digit_recongition/tensorflow/",one_hot=True)

Extracting E:/machine_data/digit_recongition/tensorflow/train-images-idx3-ubyte.gz
Extracting E:/machine_data/digit_recongition/tensorflow/train-labels-idx1-ubyte.gz
Extracting E:/machine_data/digit_recongition/tensorflow/t10k-images-idx3-ubyte.gz
Extracting E:/machine_data/digit_recongition/tensorflow/t10k-labels-idx1-ubyte.gz


In [21]:
print (mnist.train.images.shape,mnist.train.labels.shape)
print (mnist.test.images.shape,mnist.test.labels.shape)
print (mnist.validation.images.shape,mnist.validation.labels.shape)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)


In [46]:
a1=mnist.train.images
b1=mnist.train.labels

In [47]:
print (type(a1))
print (type(b1))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [73]:
print (b1[1:2])

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [14]:
sess = tf.InteractiveSession()      # 建立交互式会话
tf.initialize_all_variables().run() # 所有变量初始化
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)    # 获得一批100个数据
    train.run({x: batch_xs, y: batch_ys})   # 给训练模型提供输入和输出
print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.9137


可以看到，在模型搭建完以后，我们只要为模型提供输入和输出，模型就能够自己进行训练和测试了。中间的求导，求梯度，反向传播等等繁杂的事情，tensorflow都会帮你自动完成。

In [64]:
#测试
import tensorflow as tf
a = tf.placeholder(tf.float32)
y = tf.log(a)
sess = tf.Session() 
print (sess.run(y, feed_dict={a: [1,2]}))

[ 0.          0.69314718]
